# **Spatial Distribution for Eurasian Tree Sparow**

In [ ]:
import pandas as pd

df = pd.read_csv("birds_observation_data_with_index.csv")

totals_bird_counts = df.groupby('Date Index')['Number of Birds'].sum()
print(totals_bird_counts)


last_date_index = df['Date Index'].max()
print("Last Date Index:", last_date_index)

Date Index
1      1054
2      2637
3      1361
4       885
5       231
6       238
7       172
8        63
9        38
10       60
11      204
12      171
13       33
14      205
15      138
16       50
17      845
18     1345
19      423
20      238
21      254
22      109
23      199
24      120
25      190
26      232
27      155
28      102
29      117
30      128
31      104
32      112
33       58
34       78
35       29
36      196
37      786
38       54
39      606
40      563
41      360
42      602
43      917
44      300
45      717
46     1245
47      512
48      419
49      312
50      435
51      103
52      519
53      326
54      110
55       90
56       25
57      359
58      126
59      549
60      122
61      512
62      331
63     1061
64      645
65      519
66      164
67      342
68      123
69      204
70      201
71      261
72      572
73      931
74     1291
75     1052
76      194
77      138
78      217
79      228
80       72
81      128
82      296
83   

**GRU**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import folium
from folium.plugins import HeatMap
from IPython.display import display
from string import ascii_uppercase

# -------------------------------
# Data Preparation
# -------------------------------
df = pd.read_csv("birds_observation_data_with_index.csv")

# Find most frequently observed species
top_species = df['Bird Name'].value_counts().idxmax()
df = df[df['Bird Name'] == top_species].copy()
print(f"Using most frequent species: {top_species}")

# Calculate daily totals
daily_totals = df.groupby('Date Index')['Number of Birds'].sum()

# Feature selection
features = ['Date Index', 'Longitude', 'Latitude']
X = df[features].copy()
y = df['Number of Birds'].copy()

# Apply log1p transformation
X_log = X.copy()
X_log['Date Index'] = np.log1p(X_log['Date Index'])
X_log['Longitude'] = np.log1p(X_log['Longitude'])
X_log['Latitude'] = np.log1p(X_log['Latitude'])
y_log = np.log1p(y)

X_scaled = X_log.values
y_scaled = y_log.values

Using most frequent species: Eurasian Tree Sparrow


In [2]:
# -------------------------------
# Create Sequences
# -------------------------------
window_size = 25
def simple_sequences(data, targets, window_size):
    X_seq, y_seq = [], []
    for i in range(len(data) - window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_sequences, y_sequences = simple_sequences(X_scaled, y_scaled, window_size)
print(f"Sequences generated: {len(X_sequences)}")

# Train-test split
split_idx = int(0.8 * len(X_sequences))
X_train, X_test = X_sequences[:split_idx], X_sequences[split_idx:]
y_train, y_test = y_sequences[:split_idx], y_sequences[split_idx:]

Sequences generated: 1708


In [3]:
# -------------------------------
# GRU Model
# -------------------------------
def gru_model(input_shape):
    model = Sequential()
    model.add(GRU(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(GRU(32))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

model = gru_model((window_size, len(features)))
history = model.fit(X_train, y_train, 
                    epochs=50, batch_size=32, 
                    validation_data=(X_test, y_test),
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
                    verbose=0)

c:\Users\tsimi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.save("gru_bird_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("gru_bird_model.h5")

In [4]:
# -------------------------------
# Future Prediction
# -------------------------------
def future_counts(future_date_indexes, sample_size=150):
    all_data = []
    historical_totals = daily_totals.to_dict()

    base_pattern = {}
    for idx in future_date_indexes:
        similar_days = [k for k in historical_totals if abs(k - idx) % 12 == 0]
        base_pattern[idx] = np.mean([historical_totals[k] for k in similar_days]) if similar_days else np.mean(list(historical_totals.values()))
    
    for idx in base_pattern:
        base_pattern[idx] *= np.random.uniform(0.9, 1.1)

    for idx in future_date_indexes:
        sampled_coords = df[['Longitude', 'Latitude']].sample(sample_size, replace=True).reset_index(drop=True)
        temp_df = sampled_coords.copy()
        temp_df['Predicted Bird Count'] = 0

        for i in range(len(temp_df)):
            lat = temp_df.at[i, 'Latitude']
            lon = temp_df.at[i, 'Longitude']

            prior_df = df[(df['Latitude'] == lat) & (df['Longitude'] == lon)].sort_values('Date Index')
            if len(prior_df) >= window_size:
                last_obs = prior_df.tail(window_size)[['Date Index', 'Longitude', 'Latitude']]
                last_obs_log = last_obs.copy()
                last_obs_log['Date Index'] = np.log1p(last_obs_log['Date Index'])
                last_obs_log['Longitude'] = np.log1p(last_obs_log['Longitude'])
                last_obs_log['Latitude'] = np.log1p(last_obs_log['Latitude'])

                seq = last_obs_log.values
                pred = model.predict(seq.reshape(1, window_size, len(features)), verbose=0)[0][0]
                pred = np.expm1(pred)
                temp_df.at[i, 'Predicted Bird Count'] = max(10, pred)
            else:
                temp_df.at[i, 'Predicted Bird Count'] = y.mean()

        current_total = temp_df['Predicted Bird Count'].sum()
        if current_total > 0:
            target_total = base_pattern[idx]
            scaling_factor = target_total / current_total
            temp_df['Predicted Bird Count'] *= scaling_factor

        temp_df['YearMonth'] = f"Index {idx}"
        all_data.append(temp_df)

    return pd.concat(all_data, ignore_index=True)

# Predict future
last_index = df['Date Index'].max()
future_indexes = range(last_index + 1, last_index + 11)
future_predictions = future_counts(future_indexes)

# Display totals
totals = future_predictions.groupby('YearMonth')['Predicted Bird Count'].sum().reset_index()
print(totals)
print(f"Average prediction per index: {totals['Predicted Bird Count'].mean():.2f}")

C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[i, 'Predicted Bird Count'] = y.mean()
C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[i, 'Predicted Bird Count'] = y.mean()
C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dt

   YearMonth  Predicted Bird Count
0  Index 191             40.219904
1  Index 192             35.867008
2  Index 193            106.458109
3  Index 194            116.920743
4  Index 195             88.557278
5  Index 196             60.090623
6  Index 197             58.959930
7  Index 198             63.350557
8  Index 199             64.242432
9  Index 200             52.525509
Average prediction per index: 68.72


C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[i, 'Predicted Bird Count'] = y.mean()


In [5]:
# -------------------------------
# Heatmap Visualization
# -------------------------------
def add_colormap_legend(map_object):
    legend_html = '''
    <div style="
        position: fixed; 
        bottom: 150px; left: 50px; width: 300px; height: 30px; 
        z-index: 9999; font-size: 14px;
        background: linear-gradient(to right, navy, blue, lime, yellow, red);
        border: 1px solid grey;
        padding: 4px;
        color: black;
    ">
        <div style="text-align:center; font-weight:bold;">Number of Birds</div>
        <div style="display: flex; justify-content: space-between; font-size: 12px; margin-top: 4px; font-weight: 500;">
            <span style="font-weight: bold;">Low</span>
            <span style="font-weight: bold;">Medium</span>
            <span style="font-weight: bold;">High</span>
        </div>
    </div>
    '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

# top-left alphabet label (A, B, C, ...) 
def add_alphabet_label(map_object, label):
    html = f'''
    <div style="
        position: fixed; 
        top: 50px; left: 50px; 
        z-index: 10000;
        background-color: rgba(255,255,255,1.0);
        padding: 10px 15px;
        font-size: 80px;
        font-weight: bold;
        border: 2px solid #333;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
        {label}
    </div>
    '''
    map_object.get_root().html.add_child(folium.Element(html))

def create_predicted_heatmap(future_df, index_label):
    data = future_df[future_df['YearMonth'] == index_label]

    if data.empty:
        print(f"No data for {index_label}")
        return

    data = data[
        (data['Latitude'] >= 1) & (data['Latitude'] <= 7.5) &
        (data['Longitude'] >= 99) & (data['Longitude'] <= 120)
    ]

    norm_counts = (data['Predicted Bird Count'] - data['Predicted Bird Count'].min()) / \
                  (data['Predicted Bird Count'].max() - data['Predicted Bird Count'].min() + 1e-6)

    heat_data = data[['Latitude', 'Longitude']].copy()
    heat_data['Weight'] = norm_counts
    heat_data = heat_data.values.tolist()

    map_center = [4.210484, 107.966100]
    m = folium.Map(location=map_center, zoom_start=6)
    HeatMap(heat_data, radius=15).add_to(m)

    # Add title
    title_html = f'<h3 align="center" style="font-size:16px"><b>Heatmap for {index_label}</b></h3>'
    m.get_root().html.add_child(folium.Element(title_html))

    # Add colormap legend
    add_colormap_legend(m)

    # Add alphabet label (e.g., A, B, C... based on index position)
    # label = chr(65 + sorted(future_df['YearMonth'].unique()).index(index_label))  # ASCII A=65
    add_alphabet_label(m, index_label)

    # Display map
    display(m)


# Show maps
for idx in sorted(future_predictions['YearMonth'].unique()):
    create_predicted_heatmap(future_predictions, idx)


In [6]:
# -------------------------------
# Spatial Performance Evaluation
# -------------------------------

def evaluate_spatial_performance(model, X_test, y_test, original_df, window_size, features):
    y_pred = model.predict(X_test, verbose=0).flatten()
    
    y_pred_actual = np.expm1(y_pred)
    y_test_actual = np.expm1(y_test)
    
    # Get the corresponding locations for each test sample
    # The last window_size points in the original data before test split
    test_start_idx = len(original_df) - len(X_test) - window_size
    test_locations = original_df.iloc[test_start_idx:][['Latitude', 'Longitude']].values
    
    # the locations corresponding to the predictions (skip the first window_size)
    test_locations = test_locations[window_size:]
    
    eval_df = pd.DataFrame({
        'Latitude': test_locations[:, 0],
        'Longitude': test_locations[:, 1],
        'Actual': y_test_actual,
        'Predicted': y_pred_actual
    })
    
    # Bin locations into geographic regions for evaluation
    # create 0.5 degree bins 
    eval_df['Lat_bin'] = np.round(eval_df['Latitude'] * 2) / 2
    eval_df['Lon_bin'] = np.round(eval_df['Longitude'] * 2) / 2
    
    spatial_metrics = eval_df.groupby(['Lat_bin', 'Lon_bin']).apply(
        lambda x: pd.Series({
            'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
            'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
            'MAPE': np.mean(np.abs((x['Actual'] - x['Predicted']) / (x['Actual'] + 1e-6))) * 100,
            'Count': len(x),
            'Avg_Actual': x['Actual'].mean(),
            'Avg_Predicted': x['Predicted'].mean()
        })
    ).reset_index()
    
    # Filter out regions with too few samples
    spatial_metrics = spatial_metrics[spatial_metrics['Count'] >= 5]
    
    return spatial_metrics, eval_df

# Run the evaluation
spatial_metrics, eval_df = evaluate_spatial_performance(
    model, X_test, y_test, df, window_size, features
)

print("\nOverall Spatial Performance Metrics:")
print(f"Average RMSE across regions: {spatial_metrics['RMSE'].mean():.2f}")
print(f"Average MAE across regions: {spatial_metrics['MAE'].mean():.2f}")


Overall Spatial Performance Metrics:
Average RMSE across regions: 4.98
Average MAE across regions: 3.52


C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2585277928.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spatial_metrics = eval_df.groupby(['Lat_bin', 'Lon_bin']).apply(


**LSTM**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import folium
from folium.plugins import HeatMap
from IPython.display import display

# -------------------------------
# Data Preparation
# -------------------------------
df = pd.read_csv("birds_observation_data_with_index.csv")

# Find most frequently observed species
top_species = df['Bird Name'].value_counts().idxmax()
df = df[df['Bird Name'] == top_species].copy()
print(f"Using most frequent species: {top_species}")

# Calculate daily totals
daily_totals = df.groupby('Date Index')['Number of Birds'].sum()

# Feature selection
features = ['Date Index', 'Longitude', 'Latitude']
X = df[features].copy()
y = df['Number of Birds'].copy()

# Apply log1p transformation
X_log = X.copy()
X_log['Date Index'] = np.log1p(X_log['Date Index'])
X_log['Longitude'] = np.log1p(X_log['Longitude'])
X_log['Latitude'] = np.log1p(X_log['Latitude'])
y_log = np.log1p(y)

X_scaled = X_log.values
y_scaled = y_log.values

Using most frequent species: Eurasian Tree Sparrow


In [8]:
# -------------------------------
# Create Sequences
# -------------------------------
window_size = 20

def simple_sequences(data, targets, window_size):
    X_seq, y_seq = [], []
    for i in range(len(data) - window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_sequences, y_sequences = simple_sequences(X_scaled, y_scaled, window_size)
print(f"Sequences generated: {len(X_sequences)}")

# Train-test split
split_idx = int(0.8 * len(X_sequences))
X_train, X_test = X_sequences[:split_idx], X_sequences[split_idx:]
y_train, y_test = y_sequences[:split_idx], y_sequences[split_idx:]

Sequences generated: 1713


In [9]:
# -------------------------------
# LSTM Model
# -------------------------------

def lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

model = lstm_model((window_size, len(features)))

history = model.fit(X_train, y_train,
                    epochs=50, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
                    verbose=0)

c:\Users\tsimi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.save("lstm_bird_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("lstm_bird_model.h5")

In [10]:
# -------------------------------
# Future Prediction
# -------------------------------
def future_counts(future_date_indexes, sample_size=150):
    all_data = []
    historical_totals = daily_totals.to_dict()

    base_pattern = {}
    for idx in future_date_indexes:
        similar_days = [k for k in historical_totals if abs(k - idx) % 12 == 0]
        base_pattern[idx] = np.mean([historical_totals[k] for k in similar_days]) if similar_days else np.mean(list(historical_totals.values()))
    
    for idx in base_pattern:
        base_pattern[idx] *= np.random.uniform(0.9, 1.1)

    for idx in future_date_indexes:
        sampled_coords = df[['Longitude', 'Latitude']].sample(sample_size, replace=True).reset_index(drop=True)
        temp_df = sampled_coords.copy()
        temp_df['Predicted Bird Count'] = 0

        for i in range(len(temp_df)):
            lat = temp_df.at[i, 'Latitude']
            lon = temp_df.at[i, 'Longitude']

            prior_df = df[(df['Latitude'] == lat) & (df['Longitude'] == lon)].sort_values('Date Index')
            if len(prior_df) >= window_size:
                last_obs = prior_df.tail(window_size)[['Date Index', 'Longitude', 'Latitude']]
                last_obs_log = last_obs.copy()
                last_obs_log['Date Index'] = np.log1p(last_obs_log['Date Index'])
                last_obs_log['Longitude'] = np.log1p(last_obs_log['Longitude'])
                last_obs_log['Latitude'] = np.log1p(last_obs_log['Latitude'])

                seq = last_obs_log.values
                pred = model.predict(seq.reshape(1, window_size, len(features)), verbose=0)[0][0]
                pred = np.expm1(pred)
                temp_df.at[i, 'Predicted Bird Count'] = max(10, pred)
            else:
                temp_df.at[i, 'Predicted Bird Count'] = y.mean()

        current_total = temp_df['Predicted Bird Count'].sum()
        if current_total > 0:
            target_total = base_pattern[idx]
            scaling_factor = target_total / current_total
            temp_df['Predicted Bird Count'] *= scaling_factor

        temp_df['YearMonth'] = f"Index {idx}"
        all_data.append(temp_df)

    return pd.concat(all_data, ignore_index=True)

# Predict future
last_index = df['Date Index'].max()
future_indexes = range(last_index + 1, last_index + 11)
future_predictions = future_counts(future_indexes)

# Display totals
totals = future_predictions.groupby('YearMonth')['Predicted Bird Count'].sum().reset_index()
print(totals)
print(f"Average prediction per index: {totals['Predicted Bird Count'].mean():.2f}")

C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[i, 'Predicted Bird Count'] = y.mean()
C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[i, 'Predicted Bird Count'] = y.mean()
C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\2721742127.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.668205424120023' has dtype incompatible with int64, please explicitly cast to a compatible dt

   YearMonth  Predicted Bird Count
0  Index 191             43.662458
1  Index 192             40.555783
2  Index 193            102.655710
3  Index 194            106.141120
4  Index 195             78.095436
5  Index 196             58.603859
6  Index 197             60.111041
7  Index 198             72.978410
8  Index 199             66.917178
9  Index 200             58.171748
Average prediction per index: 68.79


In [11]:
# -------------------------------
# Heatmap Visualization
# -------------------------------
def add_colormap_legend(map_object):
    legend_html = '''
    <div style="
        position: fixed; 
        bottom: 150px; left: 50px; width: 300px; height: 30px; 
        z-index: 9999; font-size: 14px;
        background: linear-gradient(to right, navy, blue, lime, yellow, red);
        border: 1px solid grey;
        padding: 4px;
        color: black;
    ">
        <div style="text-align:center; font-weight:bold;">Number of Birds</div>
        <div style="display: flex; justify-content: space-between; font-size: 12px; margin-top: 4px; font-weight: 500;">
            <span style="font-weight: bold;">Low</span>
            <span style="font-weight: bold;">Medium</span>
            <span style="font-weight: bold;">High</span>
        </div>
    </div>
    '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

def add_index_label(map_object, label):
    html = f'''
    <div style="
        position: fixed; 
        top: 50px; left: 50px; 
        z-index: 10000;
        background-color: rgba(255,255,255,1.0);
        padding: 10px 15px;
        font-size: 80px;
        font-weight: bold;
        border: 2px solid #333;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
        {label}
    </div>
    '''
    map_object.get_root().html.add_child(folium.Element(html))

def create_predicted_heatmap(future_df, index_label):
    data = future_df[future_df['YearMonth'] == index_label]

    if data.empty:
        print(f"No data for {index_label}")
        return

    data = data[
        (data['Latitude'] >= 1) & (data['Latitude'] <= 7.5) &
        (data['Longitude'] >= 99) & (data['Longitude'] <= 120)
    ]

    norm_counts = (data['Predicted Bird Count'] - data['Predicted Bird Count'].min()) / \
                  (data['Predicted Bird Count'].max() - data['Predicted Bird Count'].min() + 1e-6)

    heat_data = data[['Latitude', 'Longitude']].copy()
    heat_data['Weight'] = norm_counts
    heat_data = heat_data.values.tolist()

    map_center = [4.210484, 107.966100]
    m = folium.Map(location=map_center, zoom_start=6)
    HeatMap(heat_data, radius=15).add_to(m)

    # Add title
    title_html = f'<h3 align="center" style="font-size:16px"><b>Heatmap for {index_label}</b></h3>'
    m.get_root().html.add_child(folium.Element(title_html))

    # Add colormap legend and label
    add_colormap_legend(m)
    add_index_label(m, index_label)

    # Display map
    display(m)


# Show maps
for idx in sorted(future_predictions['YearMonth'].unique()):
    create_predicted_heatmap(future_predictions, idx)


In [12]:
# -------------------------------
# Spatial Performance Evaluation
# -------------------------------

def evaluate_spatial_performance(model, X_test, y_test, original_df, window_size, features):
    y_pred = model.predict(X_test, verbose=0).flatten()
    
    y_pred_actual = np.expm1(y_pred)
    y_test_actual = np.expm1(y_test)
    
    # Get the corresponding locations for each test sample
    # The last window_size points in the original data before test split
    test_start_idx = len(original_df) - len(X_test) - window_size
    test_locations = original_df.iloc[test_start_idx:][['Latitude', 'Longitude']].values
    
    # the locations corresponding to the predictions (skip the first window_size)
    test_locations = test_locations[window_size:]
    
    eval_df = pd.DataFrame({
        'Latitude': test_locations[:, 0],
        'Longitude': test_locations[:, 1],
        'Actual': y_test_actual,
        'Predicted': y_pred_actual
    })
    
    # Bin locations into geographic regions for evaluation
    # create 0.5 degree bins 
    eval_df['Lat_bin'] = np.round(eval_df['Latitude'] * 2) / 2
    eval_df['Lon_bin'] = np.round(eval_df['Longitude'] * 2) / 2
    
    # Calculate metrics by region
    spatial_metrics = eval_df.groupby(['Lat_bin', 'Lon_bin']).apply(
        lambda x: pd.Series({
            'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
            'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
            'Count': len(x),
            'Avg_Actual': x['Actual'].mean(),
            'Avg_Predicted': x['Predicted'].mean()
        })
    ).reset_index()
    
    # Filter out regions with too few samples
    spatial_metrics = spatial_metrics[spatial_metrics['Count'] >= 5]
    
    return spatial_metrics, eval_df

# Run the evaluation
spatial_metrics, eval_df = evaluate_spatial_performance(
    model, X_test, y_test, df, window_size, features
)

print("\nOverall Spatial Performance Metrics:")
print(f"Average RMSE across regions: {spatial_metrics['RMSE'].mean():.2f}")
print(f"Average MAE across regions: {spatial_metrics['MAE'].mean():.2f}")


Overall Spatial Performance Metrics:
Average RMSE across regions: 4.96
Average MAE across regions: 3.51


C:\Users\tsimi\AppData\Local\Temp\ipykernel_15564\1453465919.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spatial_metrics = eval_df.groupby(['Lat_bin', 'Lon_bin']).apply(
